In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r '/content/drive/My Drive/data_enru/raw' 'data'

In [0]:
### Helper class for word indexing
SOS_token = 0 # Start of sentence
EOS_token = 1 # End of sentence

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: 'SOS', 1: 'EOS'}
        self.n_words = 2 # Initialize w/ SOS and EOS

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            # Add new word
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            # Add seen word by increasing its count
            self.word2count[word] += 1

In [0]:
### Normalize text
def unicode_to_ascii(s):
    # Convert Unicode string to plain ASCII characters
    normalized_s = [c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn']
    return ''.join(normalized_s)

def normalize_string(s):
    # Lowercase, strip whitespace, remove punctuation and non-alphabet characters
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яА-Я].!?]+", r" ", s)
    return s

In [0]:
### Parse and clean text data
def readLangs(lang1, lang2, reverse=False):

    print('Reading lines from file...')

    # Read text from file, split into lines
    lines = open('data/corpus.en_ru.1m.%s' % (lang1), encoding='utf-8').read().strip().split('\n')
    lines1 =  open('data/corpus.en_ru.1m.%s' % (lang2), encoding='utf-8').read().strip().split('\n')
    # Split lines into pairs, normalize
    pairs = [[normalize_string(s) for s in l] for l in zip(lines, lines1)]
    # pairs = [[normalize_string(s) for s in l.split('\t')] for l in lines]

    if reverse: # If we're reversing pairs
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


In [0]:
MAX_LENGTH = 20 # Max sentence length, number of words

# Filter to only use sentences starting with "I am", "She is", "They are", etc.
# eng_prefixes = (
#         'i am', 'i m',
#         'she is', 'she s',
#         'he is', 'he s',
#         'you are', 'you re',
#         'we are', 'we re',
#         'they are', 'they re'
#         )

def pair_filter(p):
    """
    Filter for pairs that fall within the MAX_LENGTH and start with our prefixes
    Returns True or False
    If X to eng/reverse=True -> p[1].startswith
    If eng to X/reverse=False -> p[0].startswith
    """
    return (len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH)

def filter_pairs(pairs):
    # Apply pair filter
    return [pair for pair in pairs if pair_filter(pair)]

### Prepare data
def prepare_data(lang1, lang2, reverse=False):
    # Read sentence pairs
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print('Read %s sentence pairs' % len(pairs))

    # Filter pairs
    pairs = filter_pairs(pairs)
    print('Filtered down to %s sentence pairs' % len(pairs))

    # Count words
    print('Counting words...')
    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])

    print('Counted words:')
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)

    return input_lang, output_lang, pairs

# Sample pairs
input_lang, output_lang, pairs = prepare_data('en', 'ru', True)
print(random.choice(pairs))

Reading lines from file...
Read 1000000 sentence pairs
Filtered down to 456370 sentence pairs
Counting words...
Counted words:
ru 441274
en 243812
['"даруй же мне, иисусе, ту веру, которую я воистину хочу .', '"grant me, oh jesus, the faith i truly desire .']


In [0]:
##### SEQ2SEQ MODEL

class EncoderRNN(nn.Module):
    """
    Seq2seq encoder is an RNN.
    For each input word, the encoder outputs a vector and a hidden state, and
    uses the hidden state for the next input word.
    """
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)

        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


class DecoderRNN(nn.Module):
    """
    Decoder is another RNN that takes in the encoder output vector(s) and
    outputs a sequence of words to create the translation.
    The most basic seq2seq decoder uses only the last output of the encoder.
    This last output is sometimes caled the "context vector", as it encodes
    the context of the entire sequence. This context vector is used as the
    initial hidden state of the decoder.
    At each step of decoding, the decoder is given an input token and hidden
    state. The initial input token is the start of string (SOS) token.
    The first hidden state is the context vector (the encoder's last hidden
    state).
    """
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))

        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


##### ATTENTION
"""
Calculate a set of attention weights.
Multiply attention weights by the encoder output vectors to create a weighted
combination. The result would contain information about that specific part of
the input sequence, and thus help the decoder choose the right output words.
To calculate the attention weights, we'll use a feed-forward layer that uses
the decoder's input and hidden state as inputs.
We will have to choose a max sentence length (input length, for encoder outputs),
wherein sentences of the max length will use all attention weights, while shorter
sentences would only use the first few.
"""

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attention = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attention_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)

        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attention_weights = F.softmax(self.attention(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attention_applied = torch.bmm(attention_weights.unsqueeze(0),
                encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attention_applied[0]), 1)
        output = self.attention_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)

        return output, hidden, attention_weights

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
##### NETWORK PREPROCESSING HELPERS
"""
Prepare training data by converting pairs into input and target tensors.
"""

def indices_from_sentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensor_from_sentence(lang, sentence):
    indices = indices_from_sentence(lang, sentence)
    indices.append(EOS_token)
    sentence_tensor = torch.tensor(indices, dtype=torch.long, device=device).view(-1, 1)

    return sentence_tensor

def tensors_from_pair(pair):
    input_tensor = tensor_from_sentence(input_lang, pair[0])
    target_tensor = tensor_from_sentence(output_lang, pair[1])

    return (input_tensor, target_tensor)

In [0]:
"""
Helper functions for printing time elapsed and estimated remaining time for
training.
"""
import time
import math

def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60

    return '%dm %ds' % (m, s)

def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s

    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))

In [0]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    # Train one interation
    encoder_hidden = encoder.init_hidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    # Encode input
    for e_i in range(input_length):
        # print(f"{e_i}/{input_length}")
        # Include hidden state from the last input when encoding current input
        encoder_output, encoder_hidden = encoder(input_tensor[e_i], encoder_hidden)
        encoder_outputs[e_i] = encoder_output[0, 0]

    # Decoder uses SOS token as first input
    decoder_input = torch.tensor([[SOS_token]], device=device)

    # Decoder uses last hidden state of encoder as first hidden state
    decoder_hidden = encoder_hidden

    # Randomly decide whether or not to use teacher forcing for decoder
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for d_i in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)

            loss += criterion(decoder_output, target_tensor[d_i])

            decoder_input = target_tensor[d_i] # Teacher forcing
    else:
        # No teacher forcing: use decoder's prediction as next input
        for d_i in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            top_v, top_i = decoder_output.topk(1)
            decoder_input = top_i.squeeze().detach() # Detach from history as input

            loss += criterion(decoder_output, target_tensor[d_i])

            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [0]:
def train_iters(encoder, decoder, n_iters, print_every=1, plot_every=100, learning_rate=0.01):
    """
    Train the network, track progress:
        - Start timer
        - Initialize optimizers and criterion
        - Create set of training pairs
        - Start empty losses array for plotting
        - Train many iterations, occasionally print progress and average loss.
    """

    start = time.time()
    plot_losses = []
    print_loss_total = 0 # Reset after each print_every
    plot_loss_total = 0 # Reset after each plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensors_from_pair(random.choice(pairs)) for i in range(n_iters)]

    criterion = nn.NLLLoss() # Negative log likelihood loss

    for i in range(1, n_iters + 1):
        training_pair = training_pairs[i - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder,
                encoder_optimizer, decoder_optimizer, criterion)

        print_loss_total += loss
        plot_loss_total += loss

        # Print progress
        if i % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0 # Reset
            print('%s (%d %d%%) %.4f' % (time_since(start, i / n_iters),
                             i, i / n_iters * 100, print_loss_avg))

    #     # Plot progress
    #     if i % plot_every == 0:
    #         plot_loss_avg = plot_loss_total / plot_every
    #         plot_losses.append(plot_loss_avg)
    #         plot_loss_total = 0 # Reset

    # show_plot(plot_losses)

In [0]:
##### PLOTTING RESULTS
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker

def show_plot(points):
    plt.figure()
    fig, ax = plt.subplots()

    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)

    plt.plot(points)
    # TODO: savefig


##### EVALUATION
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensor_from_sentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.init_hidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for e_i in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[e_i], encoder_hidden)
            encoder_outputs[e_i] += encoder_output[0, 0]

        # Start of sentence token
        decoder_input = torch.tensor([[SOS_token]], device=device)

        # Decoder's initial hidden state is encoder's last hidden state
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for d_i in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[d_i] = decoder_attention.data

            top_v, top_i = decoder_output.data.topk(1)

            if top_i.item() == EOS_token: # End of sentence
                decoded_words.append('<EOS>')
                break
            else:
                # Append prediction
                decoded_words.append(output_lang.index2word[top_i.item()])

            # Use prediction as input
            decoder_input = top_i.squeeze().detach()

        return decoded_words, decoder_attentions[:d_i + 1]


def evaluate_randomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)

        print('>', pair[0])
        print('=', pair[1])

        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)

        print('<', output_sentence)
        print()


In [0]:
hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attention_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

n_iters = 75000
train_iters(encoder, attention_decoder, n_iters, print_every=1)

0m 9s (- 11621m 6s) (1 0%) 12.3954
0m 9s (- 6091m 45s) (2 0%) 12.4189
0m 10s (- 4305m 22s) (3 0%) 12.3951
0m 10s (- 3416m 22s) (4 0%) 12.3926
0m 11s (- 2875m 4s) (5 0%) 12.3620
0m 11s (- 2455m 22s) (6 0%) 3.5502
0m 12s (- 2146m 8s) (7 0%) 12.2574
0m 12s (- 1914m 15s) (8 0%) 12.3332
0m 12s (- 1732m 41s) (9 0%) 4.9175
0m 12s (- 1600m 15s) (10 0%) 2.3120
0m 13s (- 1481m 49s) (11 0%) 3.3432
0m 13s (- 1405m 10s) (12 0%) 12.2198
0m 13s (- 1340m 11s) (13 0%) 12.2363
0m 14s (- 1265m 40s) (14 0%) 3.7060
0m 14s (- 1212m 19s) (15 0%) 12.2368
0m 14s (- 1168m 13s) (16 0%) 12.1469
0m 15s (- 1118m 14s) (17 0%) 12.1589
0m 15s (- 1070m 7s) (18 0%) 3.0892
0m 15s (- 1033m 54s) (19 0%) 1.2811
0m 16s (- 1001m 47s) (20 0%) 12.0690
0m 16s (- 967m 43s) (21 0%) 1.3703
0m 16s (- 933m 54s) (22 0%) 2.1372
0m 16s (- 907m 40s) (23 0%) 2.3913
0m 16s (- 882m 54s) (24 0%) 3.3680
0m 17s (- 861m 56s) (25 0%) 2.6630
0m 17s (- 849m 45s) (26 0%) 12.0379
0m 18s (- 838m 46s) (27 0%) 11.8803
0m 18s (- 828m 0s) (28 0%) 11.4558

KeyboardInterrupt: ignored

In [0]:
 evaluate_randomly(encoder, attention_decoder)

> - так это не правда, что он женат !
= "then it is not true that he's married !"
< the the the the the the the the the the the . <EOS>

> дмитрий медведев призвал «насыщать союзное государство реальными полномочиями» .
= dmitri medvedev urged to “delegate some real responsibilities to the union state” .
< the the the the the the the the . the . <EOS>

> в номерах имеется бесплатная бутилированная вода .
= also included are complimentary bottled water and windows that open .
< the the the the the . . . <EOS>

> сумма (млн .евро): 6,78 2 d . требуемые инвестиции, недостающие средства: d1 .
= amount (mln eur): 6,78 d . required financial assistance: dl .
< the the the the the . . the . . <EOS>

> не для школьниц в стиле набокова, которые так любят бритни .
= not for her the creepy nabokov-inspired schoolgirl-meets-slut attire so beloved of britney and her peers .
< the the the the the the the the the . the . the . <EOS>

> установление контактов и никогда не устанем связь запросов и обме

In [0]:
!pip install fastBPE sacremoses subword_nmt

     |████████████████████████████████| 870kB 9.6MB/s 
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp36-cp36m-linux_x86_64.whl size=480838 sha256=aaaac8a67ea5ab00438d4c0a80ed188d7fdc5a43325aff8443b978719297eda3
  Stored in directory: /root/.cache/pip/wheels/f3/0c/9c/fc62058b4d473a5602bcd3d3edfece796f123875379ea82d79
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=6f509544ef3e4391a85496671299f098b16cdf9973db8fb3c2d8f3323c55a2a7
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built fastBPE sacremoses


In [0]:
import torch

# List available models
torch.hub.list('pytorch/fairseq')  # [..., 'transformer.wmt16.en-de', ... ]

# Load a transformer trained on WMT'16 En-De
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.ru-en', checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt',
                       tokenizer='moses', bpe='fastbpe')
en2de.eval()  # disable dropout

# # The underlying model is available under the *models* attribute
# assert isinstance(en2de.models[0], fairseq.models.transformer.TransformerModel)

# # Move model to GPU for faster translation
# en2de.cuda()

# # Translate a sentence
# en2de.translate('Я хотел бы вновь выразить от имени всех членов Комиссии нашу признательность Совещанию государств-участников за постоянную поддержку нашей работы.Хотя отдельные новые меры кредитно-денежной политики привели к значительному улучшению ситуации с кризисом государственной задолженности, темпы оживления экономики остаются слабыми.')
# 'Hallo Welt!'

# # Batched translation
# en2de.translate(['Hello world!', 'The cat sat on the mat.'])

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /root/.cache/torch/hub/master.zip


running build_ext
cythoning fairseq/data/data_utils_fast.pyx to fairseq/data/data_utils_fast.cpp
cythoning fairseq/data/token_block_utils_fast.pyx to fairseq/data/token_block_utils_fast.cpp
building 'fairseq.libbleu' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/fairseq
creating build/temp.linux-x86_64-3.6/fairseq/clib
creating build/temp.linux-x86_64-3.6/fairseq/clib/libbleu
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -c fairseq/clib/libbleu/libbleu.cpp -o build/temp.linux-x86_64-3.6/fairseq/clib/libbleu/libbleu.o -std=c++11 -O3 -DTORCH_API_INCLUDE_EXTENSION_H -DTORCH_EXTENSION_NAME=libbleu -D_GLIBCXX_USE_CXX11_ABI=0
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master
100%|██████████| 12161762203/12161762203 [06:18<00:00, 32106086.89B/s]


KeyboardInterrupt: ignored

In [0]:
# The underlying model is available under the *models* attribute
# assert isinstance(en2de.models[0], fairseq.models.transformer.TransformerModel)

# Move model to GPU for faster translation
en2de.cuda()

# Translate a sentence
en2de.translate('Я хотел бы вновь выразить от имени всех членов Комиссии нашу признательность Совещанию государств-участников за постоянную поддержку нашей работы.Хотя отдельные новые меры кредитно-денежной политики привели к значительному улучшению ситуации с кризисом государственной задолженности, темпы оживления экономики остаются слабыми.')

'I would like to reiterate, on behalf of all members of the Commission, our gratitude to the Meeting of States Parties for its continued support for our work. Although some new monetary policy measures have led to significant improvements in the public debt crisis, the pace of economic recovery remains weak.'

In [0]:
en2de.cuda()
pp = []

In [0]:
with open("dat_part3.txt", 'r') as f, open('answer1.txt', mode='w') as out_file:
      text = f.readlines()

      for i, el in enumerate(text):
        print(f"{i}/{len(text)}")
        tx = en2de.translate(el[:-1].lower())
        pp.append(tx)


        out_file.write(f"{tx}\n")



      



In [0]:
from google.colab import files
files.download( "answer1.txt" ) 

FileNotFoundError: ignored

In [0]:
en2de.translate(pp[0])

'8. In view of the above, Japan would like to propose the following elements of the final documents of the 2015 Review Conference for further discussion by States parties.'

In [0]:
for elem in pp:

    a = en2de.translate(elem)

    